<a href="https://colab.research.google.com/github/jafcn09/IA_1/blob/main/08_01_transferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

tf.random.set_seed(42)
np.random.seed(42)

In [ ]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

4431872/4422102 [==============================] - 0s 0us/step


* X_train_A: todas las clases excepto clases 5 y 6 (sandalias y camisas)

* X_train_B: clases 5 y 6 (sandalias y camisas)

In [ ]:
def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandalias o camisas
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # indices 7, 8, 9 actualizados a 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # clasificación binaria: camisa clase 6
    return ((X[~y_5_or_6], y_A), (X[y_5_or_6], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [ ]:
print("clases del primer conjunto de datos", np.unique(y_train_A))
print("clases del segundo conjunto de datos", np.unique(y_train_B))

clases del primer conjunto de datos [0 1 2 3 4 5 6 7]
clases del segundo conjunto de datos [0. 1.]


In [ ]:
print("tamaño del primer conjunto de datos", X_train_A.shape)
print("tamaño del segundo conjunto de datos", X_train_B.shape)

tamaño del primer conjunto de datos (43986, 28, 28)
tamaño del segundo conjunto de datos (200, 28, 28)


### Entrenamiento de la primera red Deep MLP

In [ ]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

In [ ]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [ ]:
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                    validation_data=(X_valid_A, y_valid_A))

Epoch 1/20
1375/1375 [==============================] - 7s 5ms/step - loss: 0.5927 - accuracy: 0.8104 - val_loss: 0.3896 - val_accuracy: 0.8667
Epoch 2/20
1375/1375 [==============================] - 6s 4ms/step - loss: 0.3523 - accuracy: 0.8786 - val_loss: 0.3289 - val_accuracy: 0.8824
Epoch 3/20
1375/1375 [==============================] - 7s 5ms/step - loss: 0.3170 - accuracy: 0.8896 - val_loss: 0.3014 - val_accuracy: 0.8989
Epoch 4/20
1375/1375 [==============================] - 6s 4ms/step - loss: 0.2973 - accuracy: 0.8975 - val_loss: 0.2894 - val_accuracy: 0.9021
Epoch 5/20
1375/1375 [==============================] - 6s 4ms/step - loss: 0.2835 - accuracy: 0.9022 - val_loss: 0.2776 - val_accuracy: 0.9066
Epoch 6/20
1375/1375 [==============================] - 7s 5ms/step - loss: 0.2729 - accuracy: 0.9061 - val_loss: 0.2735 - val_accuracy: 0.9071
Epoch 7/20
1375/1375 [==============================] - 6s 4ms/step - loss: 0.2641 - accuracy: 0.9093 - val_loss: 0.2720 - val_accuracy:

In [ ]:
model_A.save("my_model_A.h5")


### Entrenamiento modelo B

In [ ]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))
model_B.add(keras.layers.Dense(1, activation="sigmoid"))

In [ ]:
model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [ ]:
history = model_B.fit(X_train_B, y_train_B, epochs=20,
                      validation_data=(X_valid_B, y_valid_B))

Epoch 1/20
7/7 [==============================] - 1s 44ms/step - loss: 0.9573 - accuracy: 0.4650 - val_loss: 0.6314 - val_accuracy: 0.6004
Epoch 2/20
7/7 [==============================] - 0s 31ms/step - loss: 0.5692 - accuracy: 0.7450 - val_loss: 0.4784 - val_accuracy: 0.8529
Epoch 3/20
7/7 [==============================] - 0s 31ms/step - loss: 0.4503 - accuracy: 0.8650 - val_loss: 0.4102 - val_accuracy: 0.8945
Epoch 4/20
7/7 [==============================] - 0s 18ms/step - loss: 0.3879 - accuracy: 0.8950 - val_loss: 0.3647 - val_accuracy: 0.9178
Epoch 5/20
7/7 [==============================] - 0s 18ms/step - loss: 0.3435 - accuracy: 0.9250 - val_loss: 0.3300 - val_accuracy: 0.9320
Epoch 6/20
7/7 [==============================] - 0s 18ms/step - loss: 0.3081 - accuracy: 0.9300 - val_loss: 0.3019 - val_accuracy: 0.9402
Epoch 7/20
7/7 [==============================] - 0s 18ms/step - loss: 0.2800 - accuracy: 0.9350 - val_loss: 0.2804 - val_accuracy: 0.9422
Epoch 8/20
7/7 [===========

In [ ]:
model_B.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 3ms/step - loss: 0.1408 - accuracy: 0.9705


[0.1408407837152481, 0.9704999923706055]

### Transfer learning

In [ ]:
model_A = keras.models.load_model("my_model_A.h5")
#modelo clon
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

# reusando el modelo A en B
model_B_on_A = keras.models.Sequential(model_A_clone.layers[:-1])

model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

#### congelando todas las capas de B menos la de salida

In [ ]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

In [ ]:
model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])

In [ ]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=20,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/20
7/7 [==============================] - 1s 43ms/step - loss: 0.3737 - accuracy: 0.8550 - val_loss: 0.3882 - val_accuracy: 0.8519
Epoch 2/20
7/7 [==============================] - 0s 30ms/step - loss: 0.3545 - accuracy: 0.8700 - val_loss: 0.3707 - val_accuracy: 0.8651
Epoch 3/20
7/7 [==============================] - 0s 17ms/step - loss: 0.3372 - accuracy: 0.8900 - val_loss: 0.3539 - val_accuracy: 0.8763
Epoch 4/20
7/7 [==============================] - 0s 31ms/step - loss: 0.3207 - accuracy: 0.9150 - val_loss: 0.3385 - val_accuracy: 0.8895
Epoch 5/20
7/7 [==============================] - 0s 17ms/step - loss: 0.3056 - accuracy: 0.9350 - val_loss: 0.3242 - val_accuracy: 0.8966
Epoch 6/20
7/7 [==============================] - 0s 19ms/step - loss: 0.2916 - accuracy: 0.9450 - val_loss: 0.3115 - val_accuracy: 0.9087
Epoch 7/20
7/7 [==============================] - 0s 30ms/step - loss: 0.2791 - accuracy: 0.9600 - val_loss: 0.2994 - val_accuracy: 0.9189
Epoch 8/20
7/7 [===========

In [ ]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 3ms/step - loss: 0.2044 - accuracy: 0.9670


[0.20440779626369476, 0.9670000076293945]

#### reusando todas las capas entrenadas

In [ ]:
for layer in model_B_on_A.layers[:-2]:
    layer.trainable = True

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=20,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/20
7/7 [==============================] - 1s 58ms/step - loss: 0.1619 - accuracy: 0.9800 - val_loss: 0.1714 - val_accuracy: 0.9777
Epoch 2/20
7/7 [==============================] - 0s 17ms/step - loss: 0.1358 - accuracy: 0.9950 - val_loss: 0.1507 - val_accuracy: 0.9807
Epoch 3/20
7/7 [==============================] - 0s 19ms/step - loss: 0.1175 - accuracy: 0.9950 - val_loss: 0.1340 - val_accuracy: 0.9848
Epoch 4/20
7/7 [==============================] - 0s 17ms/step - loss: 0.1031 - accuracy: 0.9950 - val_loss: 0.1214 - val_accuracy: 0.9878
Epoch 5/20
7/7 [==============================] - 0s 33ms/step - loss: 0.0918 - accuracy: 0.9950 - val_loss: 0.1114 - val_accuracy: 0.9878
Epoch 6/20
7/7 [==============================] - 0s 18ms/step - loss: 0.0829 - accuracy: 0.9950 - val_loss: 0.1035 - val_accuracy: 0.9878
Epoch 7/20
7/7 [==============================] - 0s 18ms/step - loss: 0.0759 - accuracy: 1.0000 - val_loss: 0.0969 - val_accuracy: 0.9888
Epoch 8/20
7/7 [===========

In [ ]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 2ms/step - loss: 0.0527 - accuracy: 0.9945


[0.05266788229346275, 0.9944999814033508]